### 하위폴더 파일 한곳으로 모으기

In [ ]:
import os
import shutil

def copy_files_to_destination(source_folder, destination_folder):
    # 소스 폴더의 모든 하위 폴더와 그 하위 폴더를 순회합니다.
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            source_file = os.path.join(root, file)
            destination_file = os.path.join(destination_folder, file)

            # 파일 이름이 겹치지 않도록 처리합니다.
            counter = 1
            while os.path.exists(destination_file):
                file_name, file_extension = os.path.splitext(file)
                new_file_name = f"{file_name}_{counter}{file_extension}"
                destination_file = os.path.join(destination_folder, new_file_name)
                counter += 1

            # 파일을 목적지 폴더로 복사합니다.
            shutil.copy2(source_file, destination_file)
            print(f'Copied: {source_file} to {destination_file}')

# 소스 폴더와 목적지 폴더의 경로를 입력합니다.
source_folder = r"C:\Users\LMK\Desktop\도로로드마크"  # 소스 폴더의 경로
destination_folder = r"C:\Users\LMK\Desktop\도로로드마크_전처리수합"  # 목적지 폴더의 경로

# 목적지 폴더가 존재하지 않으면 생성합니다.
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# 파일을 복사합니다.
copy_files_to_destination(source_folder, destination_folder)


### categori_id 통계 및 분류

In [ ]:
# json파일이 각각 존재
import os
import json
import shutil
from collections import defaultdict


def count_category_ids_in_jsons_and_copy(source_folder, output_folder):
    category_counts = defaultdict(int)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        for image_data in data.values():
                            if 'regions' in image_data:
                                for region_key, region in image_data['regions'].items():
                                    if 'region_attributes' in region:
                                        category_id = region['region_attributes'].get('label')
                                        if category_id is not None:
                                            category_counts[category_id] += 1

                                            category_folder = os.path.join(output_folder, f"category_{category_id}")
                                            if not os.path.exists(category_folder):
                                                os.makedirs(category_folder)

                                            original_image_filename = image_data.get("filename")
                                            if original_image_filename:
                                                original_image_path = os.path.join(root, original_image_filename)
                                                if os.path.exists(original_image_path):
                                                    destination_file_path = os.path.join(category_folder, original_image_filename)
                                                    shutil.copy(original_image_path, destination_file_path)
                                                    print(f'Copied: {original_image_path} to {destination_file_path}')
                                                else:
                                                    print(f"Image file {original_image_path} does not exist")
                                            else:
                                                print("Filename not found in JSON")
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
    return dict(category_counts)

# 기타 코드는 동일...
def save_results_to_file(results, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for category_id, count in results.items():
            f.write(f"Category ID {category_id}: {count} file(s)\n")

# 소스 폴더 경로를 설정합니다.
source_folder = r"C:\Users\LMK\Desktop\mult\multi_img\train"# 소스 폴더의 경로를 입력하세요.
output_folder = r"C:\Users\LMK\Desktop\output_multi_train"  # 결과를 저장할 폴더 경로를 입력하세요.
output_file = r"C:\Users\LMK\Desktop\category_counts_multi_train.txt"  # 결과를 저장할 파일 경로를 입력하세요.

# 카테고리 ID 갯수를 세고 결과를 출력하고 파일을 복사합니다.
category_counts = count_category_ids_in_jsons_and_copy(source_folder, output_folder)

# 결과를 파일에 저장합니다.
save_results_to_file(category_counts, output_file)

# 결과를 출력합니다.
for category_id, count in category_counts.items():
    print(f"Category ID {category_id}: {count} file(s)")


In [ ]:
# 하나의 json으로 통합

import os
import json
import shutil
from collections import defaultdict

def count_category_ids_in_single_json_and_copy(json_file, source_folder, output_folder):
    category_counts = defaultdict(int)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for image_data in data.values():
                if 'regions' in image_data:
                    for region_key, region in image_data['regions'].items():
                        if 'region_attributes' in region:
                            category_id = region['region_attributes'].get('label')
                            if category_id is not None:
                                category_counts[category_id] += 1

                                category_folder = os.path.join(output_folder, f"category_{category_id}")
                                if not os.path.exists(category_folder):
                                    os.makedirs(category_folder)

                                original_image_filename = image_data.get("filename")
                                if original_image_filename:
                                    original_image_path = os.path.join(source_folder, original_image_filename)
                                    if os.path.exists(original_image_path):
                                        destination_file_path = os.path.join(category_folder, original_image_filename)
                                        shutil.copy(original_image_path, destination_file_path)
                                        print(f'Copied: {original_image_path} to {destination_file_path}')
                                    else:
                                        print(f"Image file {original_image_path} does not exist")
                                else:
                                    print("Filename not found in JSON")
    except Exception as e:
        print(f"Error reading {json_file}: {e}")

    return dict(category_counts)

def save_results_to_file(results, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for category_id, count in results.items():
            f.write(f"Category ID {category_id}: {count} file(s)\n")

# JSON 파일과 폴더 경로를 설정합니다.
json_file = r"C:\Users\LMK\Desktop\mult\multi_img\val\val_input.json"  # JSON 파일 경로를 입력하세요.
source_folder = r"C:\Users\LMK\Desktop\mult\multi_img\val"  # 이미지 파일이 있는 폴더 경로를 입력하세요.
output_folder = r"C:\Users\LMK\Desktop\output_multi_val"  # 결과를 저장할 폴더 경로를 입력하세요.
output_file = r"C:\Users\LMK\Desktop\category_counts_multi_val.txt"  # 결과를 저장할 파일 경로를 입력하세요.

# 카테고리 ID 갯수를 세고 결과를 출력하고 파일을 복사합니다.
category_counts = count_category_ids_in_single_json_and_copy(json_file, source_folder, output_folder)

# 결과를 파일에 저장합니다.
save_results_to_file(category_counts, output_file)

# 결과를 출력합니다.
for category_id, count in category_counts.items():
    print(f"Category ID {category_id}: {count} file(s)")


### json 파일 불러오기(원본)

In [ ]:
import os
import shutil

# 이미지 폴더의 상위 경로 설정
base_image_folder = r"C:\Users\LMK\Desktop\output"
json_source_folder = r"E:\087.도로 로드마크 인식을 위한 주행 영상 데이터\01.데이터\1.Training\라벨링데이터\2D_JSON"

# 이미지 파일 확장자 목록 (필요에 따라 확장자를 추가)
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# 상위 이미지 폴더를 스캔하여 모든 하위 폴더를 검색
for root, dirs, files in os.walk(base_image_folder):
    for file in files:
        # 이미지 파일인지 확인
        if os.path.splitext(file)[1].lower() in image_extensions:
            image_name_without_extension = os.path.splitext(file)[0]
            json_file_name = image_name_without_extension + '.json'
            json_source_path = os.path.join(json_source_folder, json_file_name)
            json_target_path = os.path.join(root, json_file_name)  # 동일한 폴더로 복사

            # JSON 파일이 소스 폴더에 있는지 확인
            if os.path.exists(json_source_path):
                shutil.copy(json_source_path, json_target_path)  # JSON 파일을 동일한 폴더로 복사
               # print(f"Copied: {json_source_path} -> {json_target_path}")
            else:
                print(f"JSON file not found for image: {file}")


### 필요클래스 추리기

In [ ]:
import os
import json

# 수정할 코드 목록
code_to_detect = [389, 391, 402, 403, 412, 432]

    # 389: 주차금지 - seg
    # 391: 정차, 주차금지 - seg
    # 402: 속도제한 어린이 보호 구역 - bbox
    # 403: 어린이 보호 구역 - bbox
    # 412: 횡단보도 - seg
    # 432: 정차금지지대 -seg

    # 클래스별 개수 찾기


# 상위 폴더 경로 설정
base_folder = r"C:\Users\LMK\Desktop\6CLASS" # 이미지 및 JSON 파일이 들어있는 상위 폴더의 경로

# 폴더 탐색
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith('.json'):
            json_file_path = os.path.join(root, file)
            with open(json_file_path, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)

            # 'categories' 항목 제거
            if 'categories' in data:
                del data['categories']

            # 'images' 항목 수정
            if 'images' in data:
                for image in data['images']:
                    image_keys_to_keep = {'filename', 'width', 'height'}
                    keys_to_remove = set(image.keys()) - image_keys_to_keep
                    for key in keys_to_remove:
                        del image[key]

            # 'annotations' 항목 수정
            if 'annotations' in data:
                data['annotations'] = [annotation for annotation in data['annotations'] if annotation['category_id'] in code_to_detect]

            # 수정된 JSON 파일 저장
            with open(json_file_path, 'w', encoding='utf-8') as json_file:
                json.dump(data, json_file, indent=4, ensure_ascii=False)
            #print(f"Updated {json_file_path}")



### json 수정 -> 402, 403제외하고 bbox 제거

In [ ]:
import os
import json

# 상위 폴더 경로 설정
base_folder = r"C:\Users\LMK\Desktop\6class_merge"  # JSON 파일이 들어있는 상위 폴더

# 제외할 카테고리 ID 설정
categories_to_exclude = [402, 403]

# 폴더 탐색
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith('.json'):
            json_file_path = os.path.join(root, file)

            try:
                # JSON 파일 읽기
                with open(json_file_path, 'r', encoding='utf-8') as json_file:
                    data = json.load(json_file)

                # 'annotations' 항목 수정
                for annotation in data.get('annotations', []):
                    category_id = annotation.get('category_id', 'Unknown')
                    if category_id not in categories_to_exclude:
                        # bbox 제거
                        if 'bbox' in annotation:
                            del annotation['bbox']

                # 수정된 JSON 파일 저장
                with open(json_file_path, 'w', encoding='utf-8') as json_file:
                    json.dump(data, json_file, indent=4, ensure_ascii=False)

                print(f"Processed {json_file_path}")

            except Exception as e:
                print(f"Error processing file {json_file_path}: {e}")


### class 이름 변경

In [ ]:
import os
import json

# 상위 폴더 경로 설정
base_folder = r"C:\Users\LMK\Desktop\6class_merge"  # JSON 파일이 들어있는 상위 폴더

# 카테고리 아이디를 영어 이름으로 매핑
category_mapping = {
    402: 'Road_Speed_Limit_in_School_Zone',  # 어린이 보호 구역 속도 제한
    403: 'Road_School_Zone',                 # 어린이 보호 구역
    412: 'Crosswalk',                        # 횡단보도
    432: 'Road_No_Stopping_Zone',            # 정차 금지
    389: 'Road_No_Parking',                  # 주차 금지
    391: 'Road_No_Stopping_or_Parking'       # 정차 및 주차 금지
}

# 폴더 탐색
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith('.json'):
            json_file_path = os.path.join(root, file)

            try:
                # JSON 파일 읽기
                with open(json_file_path, 'r', encoding='utf-8') as json_file:
                    data = json.load(json_file)

                # 'annotations' 항목 수정
                for annotation in data.get('annotations', []):
                    category_id = annotation.get('category_id', None)
                    if category_id in category_mapping:
                        annotation['category_id'] = category_mapping[category_id]

                # 수정된 JSON 파일 저장
                with open(json_file_path, 'w', encoding='utf-8') as json_file:
                    json.dump(data, json_file, indent=4, ensure_ascii=False)

                print(f"Processed {json_file_path}")

            except Exception as e:
                print(f"Error processing file {json_file_path}: {e}")


### 통계

In [ ]:
import os
import json
from collections import defaultdict

# 상위 폴더 경로 설정
base_folder = r"C:\Users\LMK\Desktop\6class_merge"  # JSON 파일이 들어있는 상위 폴더
stats_file = r"C:\Users\LMK\Desktop\category_stats.json"  # 통계 파일을 저장할 경로

# 카테고리 매핑 설정
category_mapping = {
    'Road_Speed_Limit_in_School_Zone': '어린이 보호 구역 속도 제한',  # 매핑 설명 (한국어 설명)
    'Road_School_Zone': '어린이 보호 구역',
    'Crosswalk': '횡단보도',
    'Road_No_Stopping_Zone': '정차 금지',
    'Road_No_Parking': '주차 금지',
    'Road_No_Stopping_or_Parking': '정차 및 주차 금지'
}

# 카테고리별 통계 초기화
category_stats = defaultdict(int)

# 폴더 탐색
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith('.json'):
            json_file_path = os.path.join(root, file)

            try:
                # JSON 파일 읽기
                with open(json_file_path, 'r', encoding='utf-8') as json_file:
                    data = json.load(json_file)

                # JSON 파일 형식 및 데이터 확인
                if 'annotations' not in data:
                    print(f"Annotations key not found in {json_file_path}")
                    continue

                annotations = data['annotations']
                if not annotations:
                    print(f"No annotations found in {json_file_path}")
                    continue

                # 'annotations' 항목에서 카테고리 아이디를 카운트
                for annotation in annotations:
                    category_id = annotation.get('category_id')
                    if category_id is None:
                        print(f"Category ID missing in {json_file_path}")
                        continue

                    if category_id in category_mapping:
                        category_stats[category_id] += 1
                    else:
                        print(f"Unknown category ID {category_id} in {json_file_path}")

            except Exception as e:
                print(f"Error processing file {json_file_path}: {e}")

# 카테고리 이름을 추가하여 통계 생성
category_stats_with_names = {
    category_mapping[cat_id]: count for cat_id, count in category_stats.items()
}

# 통계 결과를 JSON 파일로 저장
with open(stats_file, 'w', encoding='utf-8') as json_file:
    json.dump(category_stats_with_names, json_file, indent=4, ensure_ascii=False)

print(f"Category statistics saved to {stats_file}")


### coco?-> via 변환

In [ ]:
import os
import json

def convert_to_via_format(input_folder, output_file):
    via_data = {}

    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.json'):
                json_file_path = os.path.join(root, file)

                try:
                    with open(json_file_path, 'r', encoding='utf-8') as json_file:
                        data = json.load(json_file)

                    # 이미지 파일 정보
                    image_info = data['images'][0]
                    filename = image_info['filename']
                    size = os.path.getsize(os.path.join(root, filename))
                    image_id = image_info.get('id', filename)

                    via_data[filename] = {
                        "filename": filename,
                        "size": size,
                        "regions": {},
                        "file_attributes": {}
                    }

                    # 객체 정보
                    regions = {}
                    for idx, annotation in enumerate(data['annotations']):
                        category_id = annotation.get('category_id', 'Unknown')
                        shape_type = 'polygon'
                        all_points_x = []
                        all_points_y = []

                        # Segmentation 처리
                        if 'segmentation' in annotation:
                            points = annotation['segmentation'][0]
                            for i in range(0, len(points), 2):
                                all_points_x.append(int(points[i]))
                                all_points_y.append(int(points[i+1]))

                        # Polyline 처리 (폴리곤으로 변환)
                        elif 'polyline' in annotation:
                            points = annotation['polyline'][0]
                            for i in range(0, len(points), 2):
                                all_points_x.append(int(points[i]))
                                all_points_y.append(int(points[i+1]))
                            # 첫 번째 점을 추가하여 폴리곤으로 변환
                            all_points_x.append(all_points_x[0])
                            all_points_y.append(all_points_y[0])

                        # BBox 처리 (폴리곤으로 변환)
                        elif 'bbox' in annotation:
                            x, y, width, height = annotation['bbox']
                            all_points_x = [int(x), int(x + width), int(x + width), int(x), int(x)]
                            all_points_y = [int(y), int(y), int(y + height), int(y + height), int(y)]

                        # Region 구성
                        regions[str(idx)] = {
                            "shape_attributes": {
                                "name": shape_type,
                                "all_points_x": all_points_x,
                                "all_points_y": all_points_y
                            },
                            "region_attributes": {
                                "category": category_id
                            }
                        }

                    via_data[filename]["regions"] = regions

                except Exception as e:
                    print(f"Error processing file {json_file_path}: {e}")

    # VIA 형식의 JSON 파일 저장
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(via_data, json_file, indent=4, ensure_ascii=False)

    print(f"VIA formatted data saved to {output_file}")

# 사용 예시
input_folder = r"C:\Users\LMK\Desktop\6class_merge"
output_file = r"C:\Users\LMK\Desktop\via_format1.json"
convert_to_via_format(input_folder, output_file)


In [ ]:
#폴리라인 포함 -> json병합


import os
import json

def convert_to_via_format(input_folder, output_file):
    via_data = {}

    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.json'):
                json_file_path = os.path.join(root, file)

                try:
                    with open(json_file_path, 'r', encoding='utf-8') as json_file:
                        data = json.load(json_file)

                    # 이미지 파일 정보
                    image_info = data['images'][0]
                    filename = image_info['filename']
                    size = os.path.getsize(os.path.join(root, filename))
                    image_id = image_info.get('id', filename)

                    via_data[filename] = {
                        "filename": filename,
                        "size": size,
                        "regions": {},
                        "file_attributes": {}
                    }

                    # 객체 정보
                    regions = {}
                    for idx, annotation in enumerate(data['annotations']):
                        category_id = annotation.get('label', 'Unknown')
                        all_points_x = []
                        all_points_y = []

                        # Segmentation 처리 (폴리곤)
                        if 'segmentation' in annotation:
                            shape_type = 'polygon'
                            points = annotation['segmentation'][0]
                            for i in range(0, len(points), 2):
                                all_points_x.append(int(points[i]))
                                all_points_y.append(int(points[i + 1]))
                            # Ensure the polygon is closed
                            if all_points_x[-1] != all_points_x[0] or all_points_y[-1] != all_points_y[0]:
                                all_points_x.append(all_points_x[0])
                                all_points_y.append(all_points_y[0])

                        # BBox 처리 (폴리곤으로 변환)
                        elif 'bbox' in annotation:
                            shape_type = 'polygon'
                            x, y, width, height = annotation['bbox']
                            all_points_x = [int(x), int(x + width), int(x + width), int(x), int(x)]
                            all_points_y = [int(y), int(y), int(y + height), int(y + height), int(y)]
                            # Ensure the polygon is closed
                            if all_points_x[-1] != all_points_x[0] or all_points_y[-1] != all_points_y[0]:
                                all_points_x.append(all_points_x[0])
                                all_points_y.append(all_points_y[0])

                        # 폴리라인 처리
                        elif 'polyline' in annotation:
                            shape_type = 'polyline'
                            points = annotation['polyline']
                            if isinstance(points[0], list):
                                # 여러 개의 polyline이 있는 경우
                                for sublist in points:
                                    for i in range(0, len(sublist), 2):
                                        all_points_x.append(int(sublist[i]))
                                        all_points_y.append(int(sublist[i + 1]))
                            else:
                                # 단일 polyline인 경우
                                for i in range(0, len(points), 2):
                                    all_points_x.append(int(points[i]))
                                    all_points_y.append(int(points[i + 1]))

                        # Region 구성
                        regions[str(idx)] = {
                            "shape_attributes": {
                                "name": shape_type,
                                "all_points_x": all_points_x,
                                "all_points_y": all_points_y
                            },
                            "region_attributes": {
                                "category": category_id
                            }
                        }

                    via_data[filename]["regions"] = regions

                except Exception as e:
                    print(f"Error processing file {json_file_path}: {e}")

    # VIA 형식의 JSON 파일 저장
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(via_data, json_file, indent=4, ensure_ascii=False)

    print(f"VIA formatted data saved to {output_file}")

# 사용 예시
input_folder = r"C:\Users\LMK\Desktop\어린이보호구역표지판"
output_file = r"C:\Users\LMK\Desktop\new_stats\school_zone.stats"
convert_to_via_format(input_folder, output_file)


### via 여러개 -> 한개의 json으로 합병

In [ ]:
#구분자가 있을떄

import json
import os

# JSON 파일이 있는 디렉토리 경로
json_dir = r"C:\Users\LMK\Desktop\new_stats\merge" # 경로를 파일들이 저장된 위치로 변경하세요

# 결합된 데이터를 저장할 빈 딕셔너리 초기화
combined_data = {}

# 디렉토리 내 모든 파일을 순회
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        filepath = os.path.join(json_dir, filename)
        with open(filepath, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # 파일 이름을 키로 사용하여 데이터 추가
            combined_data[filename] = data

# 결합된 데이터를 새로운 JSON 파일로 출력
output_filepath = r"C:\Users\LMK\Desktop\new_stats\merge\new_merge_10.json" # 저장할 경로를 업데이트하세요
with open(output_filepath, 'w', encoding='utf-8') as outfile:
    json.dump(combined_data, outfile, indent=4, ensure_ascii=False)

print(f"결합된 JSON 데이터가 {output_filepath}에 저장되었습니다.")


In [ ]:
##구분자 없을때 일단 파일명으로 -> 추후에 메모장에서 모두바꾸기할것

import os
import json

def merge_via_json_files(input_folder, output_file):
    merged_via_data = {}
    duplicate_files = {}

    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.json'):
                json_file_path = os.path.join(root, file)

                try:
                    with open(json_file_path, 'r', encoding='utf-8') as json_file:
                        data = json.load(json_file)

                        # 데이터가 dict인지 확인
                        if not isinstance(data, dict):
                            print(f"Warning: File {json_file_path} does not contain a valid JSON object.")
                            continue

                        for original_filename, image_data in data.items():
                            # 파일명 구분자를 JSON 파일명으로 변경
                            filename = os.path.basename(original_filename)

                            if filename in merged_via_data:
                                # 중복된 파일명 처리
                                if filename not in duplicate_files:
                                    duplicate_files[filename] = []
                                duplicate_files[filename].append(json_file_path)
                            else:
                                if not isinstance(image_data, dict):
                                    print(f"Warning: Data for {filename} in {json_file_path} is not a valid dictionary.")
                                    continue

                                # 정해진 형식에 맞추어 데이터 추가
                                merged_via_data[filename] = {
                                    "filename": image_data.get("filename", filename),
                                    "size": image_data.get("size", 0),
                                    "regions": image_data.get("regions", {}),
                                    "file_attributes": image_data.get("file_attributes", {})
                                }

                except json.JSONDecodeError as jde:
                    print(f"JSON decode error in file {json_file_path}: {jde}")
                except Exception as e:
                    print(f"Error processing file {json_file_path}: {e}")

    # 중복된 파일명 출력
    if duplicate_files:
        print("중복된 파일명이 발견되었습니다. 첫 번째 파일만 병합됩니다:")
        for filename, paths in duplicate_files.items():
            print(f"파일명: {filename}")
            for path in paths:
                print(f"  - 경로: {path}")

    # 병합된 VIA 형식의 JSON 파일 저장
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(merged_via_data, json_file, indent=4, ensure_ascii=False)

    print(f"VIA formatted data merged and saved to {output_file}")

# 사용 예시
input_folder = r"C:\Users\LMK\Desktop\어린이보호구역표지판"
output_file = r"C:\Users\LMK\Desktop\new_stats\merged_school_zone.json"
merge_via_json_files(input_folder, output_file)


In [ ]:
import os
import json
from collections import defaultdict

# 입력 및 출력 파일 경로 설정
input_via_file =  r"C:\Users\LMK\Desktop\merge_10class\merged_class_10_updated.json" # 변환된 VIA 파일 경로
stats_output_file =  r"C:\Users\LMK\Desktop\merge_10class\merged_class_10_updated_stats.json"  # 통계 결과를 저장할 경로

# 카테고리별 통계 초기화
category_stats = defaultdict(int)
total_images = 0

# VIA 파일 읽기
try:
    with open(input_via_file, 'r', encoding='utf-8') as via_file:
        via_data = json.load(via_file)

    # 총 이미지 수 계산
    total_images = len(via_data)

    # 카테고리별 통계 계산
    for image_data in via_data.values():
        regions = image_data.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('label', 'Unknown')  ##label과 CATEGORY
            category_stats[category] += 1

    # 통계 결과 출력
    print(f"Total number of images: {total_images}")
    print("Category statistics:")
    for category, count in category_stats.items():
        print(f"{category}: {count}")

    # 통계 결과를 JSON 파일로 저장
    with open(stats_output_file, 'w', encoding='utf-8') as stats_file:
        json.dump({
            "total_images": total_images,
            "category_statistics": category_stats
        }, stats_file, indent=4, ensure_ascii=False)

    print(f"Statistics saved to {stats_output_file}")

except Exception as e:
    print(f"Error processing VIA file: {e}")


Total number of images: 16276
Category statistics:
no_parking: 4307
school_zone: 892
stop: 2532
fire_hydrant: 1524
Road_No_Stopping_Zone: 565
Road_No_Stopping_or_Parking: 8975
Crosswalk: 3924
Road_No_Parking: 3407
Road_Speed_Limit_in_School_Zone: 1506
Road_School_Zone: 2050
Statistics saved to C:\Users\LMK\Desktop\merge_10class\merged_class_10_updated_stats.json


### 시각화

In [ ]:
#bbox, seg, polyline 확인용

import os
import json
import cv2
import numpy as np

# 상위 폴더 경로 설정
base_folder = r"C:\Users\LMK\Desktop\output"  # 이미지와 JSON 파일이 들어있는 상위 폴더
output_folder = r"C:\Users\LMK\Desktop\output_visual"  # 시각화된 이미지를 저장할 폴더

# 색상 설정
color_bbox = (0, 255, 0)  # 녹색
color_segmentation = (255, 0, 0)  # 빨강색
color_polyline = (0, 0, 255)  # 파란색
font_color = (255, 255, 255)  # 흰색
font_scale = 0.5
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX

# 폴더 탐색
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith('.json'):
            try:
                json_file_path = os.path.join(root, file)

                # 이미지 파일 경로 설정
                image_filename = file.replace('.json', '.png')
                image_file_path = os.path.join(root, image_filename)

                # 이미지가 존재하는지 확인
                if not os.path.exists(image_file_path):
                    print(f"Image file not found for {json_file_path}")
                    continue

                # JSON 파일 읽기
                try:
                    with open(json_file_path, 'r', encoding='utf-8') as json_file:
                        data = json.load(json_file)
                except Exception as e:
                    print(f"Error reading JSON file {json_file_path}: {e}")
                    continue

                # 이미지 읽기
                try:
                    image = cv2.imread(image_file_path)
                    if image is None:
                        raise ValueError("Image is None")
                except Exception as e:
                    print(f"Error reading image {image_file_path}: {e}")
                    continue

                # 시각화 작업
                try:
                    for annotation in data.get('annotations', []):
                        category_id = annotation.get('category_id', 'Unknown')
                        text = f"Class ID: {category_id}"

                        # bbox 시각화
                        bbox = annotation.get('bbox', [])
                        if bbox:
                            x, y, width, height = bbox
                            start_point = (int(x), int(y))
                            end_point = (int(x + width), int(y + height))
                            image = cv2.rectangle(image, start_point, end_point, color_bbox, thickness)  # 녹색
                            cv2.putText(image, f"bbox: {bbox}", (int(x), int(y) - 10), font, font_scale, font_color, thickness, cv2.LINE_AA)

                        # segmentation 시각화
                        segmentation = annotation.get('segmentation', [])
                        if segmentation:
                            points = np.array(segmentation[0], dtype=np.int32).reshape((-1, 1, 2))
                            image = cv2.polylines(image, [points], isClosed=True, color=color_segmentation, thickness=thickness)  # 빨강색
                            cv2.putText(image, "segmentation", (int(points[0][0][0]), int(points[0][0][1]) - 10), font, font_scale, font_color, thickness, cv2.LINE_AA)

                        # polyline 시각화 및 폴리곤 변환
                        polyline = annotation.get('polyline', [])
                        if polyline:
                            polyline_points = np.array(polyline[0], dtype=np.int32).reshape((-1, 2))
                            polyline_polygon_points = np.vstack((polyline_points, polyline_points[0]))  # 첫 번째 점을 추가하여 폴리곤으로 변환
                            image = cv2.polylines(image, [polyline_polygon_points], isClosed=True, color=color_polyline, thickness=thickness)  # 파란색
                            cv2.putText(image, "polyline", (int(polyline_points[0][0]), int(polyline_points[0][1]) - 10), font, font_scale, font_color, thickness, cv2.LINE_AA)

                        # 클래스 아이디 표시
                        x_pos = int(bbox[0]) if bbox else int(points[0][0][0]) if segmentation else int(polyline_points[0][0])
                        y_pos = int(bbox[1]) if bbox else int(points[0][0][1]) if segmentation else int(polyline_points[0][1])
                        cv2.putText(image, text, (x_pos, y_pos - 20), font, font_scale, font_color, thickness, cv2.LINE_AA)

                        # 각 클래스별 아웃풋 폴더 경로 설정
                        class_output_folder = os.path.join(output_folder, str(category_id))
                        os.makedirs(class_output_folder, exist_ok=True)

                        # 결과 저장
                        output_image_path = os.path.join(class_output_folder, f"{os.path.splitext(file)[0]}.png")
                        cv2.imwrite(output_image_path, image)
                        print(f"Saved visualized image to {output_image_path}")
                except Exception as e:
                    print(f"Error processing annotation in {json_file_path}: {e}")

            except Exception as e:
                print(f"Error processing file {json_file_path}: {e}")


In [ ]:
#via 한개로 시각

import os
import json
import cv2
import numpy as np

# 입력 및 출력 경로 설정
via_file = r"C:\Users\LMK\Desktop\via_format_polyline.json"  # 변환된 VIA 파일 경로
image_folder = r"C:\Users\LMK\Desktop\6class_merge"  # 원본 이미지 파일이 들어있는 폴더
output_folder = r"C:\Users\LMK\Desktop\visual_image_6merge_polyline"  # 시각화된 이미지를 저장할 폴더

# 색상 및 폰트 설정
color_mapping = {
    'Road_Speed_Limit_in_School_Zone': (0, 255, 0),  # 녹색
    'Road_School_Zone': (255, 0, 0),                 # 빨강색
    'Crosswalk': (0, 0, 255),                        # 파란색
    'Road_No_Stopping_Zone': (255, 255, 0),          # 노랑색
    'Road_No_Parking': (255, 0, 255),                # 보라색
    'Road_No_Stopping_or_Parking': (0, 255, 255)     # 청록색
}
font_color = (255, 255, 255)  # 흰색
font_scale = 0.5
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX

# VIA 파일 읽기
try:
    with open(via_file, 'r', encoding='utf-8') as via_json_file:
        via_data = json.load(via_json_file)

    # 폴더 생성
    os.makedirs(output_folder, exist_ok=True)

    # 이미지 파일 탐색 및 시각화
    for image_key, image_data in via_data.items():
        image_path = os.path.join(image_folder, image_key)
        if not os.path.exists(image_path):
            print(f"Image file not found: {image_path}")
            continue

        # 이미지 읽기
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to read image: {image_path}")
            continue

        # 객체 시각화
        regions = image_data.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('category', 'Unknown')
            if category not in color_mapping:
                print(f"Unknown category: {category}")
                continue

            color = color_mapping[category]
            all_points_x = region['shape_attributes'].get('all_points_x', [])
            all_points_y = region['shape_attributes'].get('all_points_y', [])

            if all_points_x and all_points_y and len(all_points_x) == len(all_points_y):
                points = np.array([[x, y] for x, y in zip(all_points_x, all_points_y)], dtype=np.int32)
                points = points.reshape((-1, 1, 2))

                # 폴리곤 그리기
                image = cv2.polylines(image, [points], isClosed=False, color=color, thickness=thickness)

                # 텍스트 표시 (카테고리 이름)
                text_position = (all_points_x[0], all_points_y[0] - 10)
                cv2.putText(image, category, text_position, font, font_scale, font_color, thickness, cv2.LINE_AA)

        # 결과 이미지 저장
        output_image_path = os.path.join(output_folder, image_key)
        cv2.imwrite(output_image_path, image)
        print(f"Saved visualized image to {output_image_path}")

except Exception as e:
    print(f"Error processing VIA file: {e}")


### train/val 나누기. via.json으로 나누기

In [ ]:
import os
import json
import random
from collections import defaultdict

# 설정 경로
via_file = r"C:\Users\LMK\Desktop\via_format_polyline.json"  # VIA 파일 경로
output_folder = r"C:\Users\LMK\Desktop\dataset_split2"  # 데이터셋 분할 결과 저장 폴더

# 클래스별 목표 개수
target_train_size = 250
target_val_size = 100

# 클래스별 최대 개수 설정 (실제 데이터 개수 고려)
class_target_sizes = {
    'Road_No_Stopping_or_Parking': {'train': target_train_size, 'val': target_val_size},
    'Crosswalk': {'train': target_train_size, 'val': target_val_size},
    'Road_Speed_Limit_in_School_Zone': {'train': target_train_size, 'val': target_val_size},
    'Road_School_Zone': {'train': target_train_size, 'val': target_val_size},
    'Road_No_Stopping_Zone': {'train': 450, 'val': 100},
    'Road_No_Parking': {'train': target_train_size, 'val': target_val_size}
}

# 클래스별 이미지 목록
class_images = defaultdict(list)

# VIA 파일 읽기
try:
    with open(via_file, 'r', encoding='utf-8') as f:
        via_data = json.load(f)

    # 이미지별 클래스 수집
    for image_id, image_info in via_data.items():
        regions = image_info.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('category')
            if category:
                class_images[category].append(image_id)
except Exception as e:
    print(f"Error reading VIA file: {e}")

# train과 val 세트
train_set = set()
val_set = set()

# 클래스별로 train과 val로 나누기
for category, images in class_images.items():
    random.shuffle(images)
    target_sizes = class_target_sizes.get(category, {'train': target_train_size, 'val': target_val_size})
    train_count = target_sizes['train']
    val_count = target_sizes['val']

    # 현재 카테고리에서 사용될 이미지 필터링
    filtered_images = [img for img in images if img not in train_set and img not in val_set]

    # train과 val 세트로 나누기
    train_images = filtered_images[:train_count]
    val_images = filtered_images[train_count:train_count + val_count]

    train_set.update(train_images)
    val_set.update(val_images)

# 중복 제거
val_set.difference_update(train_set)  # train_set과 중복된 항목 제거

# 결과 출력
print(f"Train set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")

# 분할된 데이터셋 저장
os.makedirs(output_folder, exist_ok=True)

train_file = os.path.join(output_folder, 'train.json')
val_file = os.path.join(output_folder, 'val.json')

# 저장할 데이터 생성
train_data = {image_id: via_data[image_id] for image_id in train_set}
val_data = {image_id: via_data[image_id] for image_id in val_set}

try:
    with open(train_file, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, indent=4, ensure_ascii=False)

    with open(val_file, 'w', encoding='utf-8') as f:
        json.dump(val_data, f, indent=4, ensure_ascii=False)

    print(f"Train set saved to {train_file}")
    print(f"Validation set saved to {val_file}")
except Exception as e:
    print(f"Error saving dataset: {e}")


Train set size: 0
Validation set size: 0
Train set saved to C:\Users\LMK\Desktop\merge_10class\train.json
Validation set saved to C:\Users\LMK\Desktop\merge_10class\val.json


In [ ]:
import os
import json

# JSON 파일 경로
json_file_path = r"C:\Users\LMK\Desktop\merge_10class\merged_class_10_updated.json"
# 원본 이미지 폴더 경로
image_folder_path = r"C:\Users\LMK\Desktop\merge"

# JSON 파일 읽기
with open(json_file_path, 'r') as json_file:
    json_data = json.load(json_file)
    # JSON 데이터가 via 형식의 딕셔너리 구조인 경우
    json_image_files = list(json_data.keys())

# 원본 이미지 목록 가져오기
image_files = [file for file in os.listdir(image_folder_path) if file.endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif'))]

# JSON에 있지만 원본 이미지 폴더에 없는 파일 찾기
missing_in_images = [file for file in json_image_files if file not in image_files]

# 원본 이미지 폴더에 있지만 JSON에 없는 파일 찾기
missing_in_json = [file for file in image_files if file not in json_image_files]

# 결과 출력
print(f"JSON 파일 내 이미지 개수: {len(json_image_files)}")
print(f"원본 이미지 폴더 내 이미지 개수: {len(image_files)}")
print(f"JSON에 있지만 원본 이미지 폴더에 없는 파일 수: {len(missing_in_images)}")
print(f"원본 이미지 폴더에 있지만 JSON에 없는 파일 수: {len(missing_in_json)}")
print(f"JSON에 있지만 원본 이미지 폴더에 없는 파일: {missing_in_images}")
print(f"원본 이미지 폴더에 있지만 JSON에 없는 파일: {missing_in_json}")


JSON 파일 내 이미지 개수: 16276
원본 이미지 폴더 내 이미지 개수: 16276
JSON에 있지만 원본 이미지 폴더에 없는 파일 수: 0
원본 이미지 폴더에 있지만 JSON에 없는 파일 수: 0
JSON에 있지만 원본 이미지 폴더에 없는 파일: []
원본 이미지 폴더에 있지만 JSON에 없는 파일: []


In [ ]:
### 이미지원본이 없는 거 json에서 삭제

import json
import shutil
import os

# JSON 파일 경로
json_file_path = r"C:\Users\LMK\Desktop\merge_10class\patch_merged_class_10.json"
# 백업 파일 경로
backup_folder_path = r"C:\Users\LMK\Desktop\backup"
os.makedirs(backup_folder_path, exist_ok=True)
backup_file_path = os.path.join(backup_folder_path, "merged_class_10_backup.json")

# JSON 파일 백업
try:
    shutil.copy(json_file_path, backup_file_path)
    print(f"원본 JSON 파일의 백업이 '{backup_file_path}'에 저장되었습니다.")
except Exception as e:
    print(f"백업 과정에서 오류가 발생했습니다: {e}")

# JSON 파일 읽기
try:
    with open(json_file_path, 'r') as json_file:
        json_data = json.load(json_file)
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {json_file_path}")
    exit(1)
except json.JSONDecodeError:
    print(f"JSON 파일을 파싱하는 중에 오류가 발생했습니다: {json_file_path}")
    exit(1)

# 제거할 이미지 파일 리스트
missing_in_images = [
    # 파일 목록을 여기에 추가해야 합니다.
    # 예: "image1.jpg", "image2.png"
 '1447846_614(1).jpg', '1448022_614(1).jpg', '1448024_614(1).jpg', '1448027_614(1).jpg', '1448031_614(1).jpg', '1448219_614(1).jpg', '1449809_614(1).jpg', '1449951_614(1).jpg', '1451944_614(1).jpg', '1455805_614(1).jpg', '968982_614.jpg', '969001_614.jpg', '969017_614.jpg', '969434_614.jpg', '969451_614.jpg', '969454_614.jpg', '970392_614.jpg', '970430_614.jpg', '970448_614.jpg', '970748_614.jpg', '970774_614.jpg', '971922_614.jpg', '971941_614.jpg', '971949_614.jpg', '971958_614.jpg', '971960_614.jpg', '971968_614.jpg', '971973_614.jpg', '971991_614.jpg', '971994_614.jpg', '971995_614.jpg', '971996_614.jpg', '972008_614.jpg', '972010_614.jpg', '972157_614.jpg', '972162_614.jpg', '973333_614.jpg', '973336_614.jpg', '973550_614.jpg', '973569_614.jpg', '973574_614.jpg', '973577_614.jpg', '973613_614.jpg', '973615_614.jpg', '973626_614.jpg', '973852_614.jpg', '973916_614.jpg', '973929_614.jpg', '973941_614.jpg', '973951_614.jpg', '974639_614.jpg', '974652_614.jpg', '975125_614.jpg', '975129_614.jpg', '975138_614.jpg', '975145_614.jpg', '975148_614.jpg', '975152_614.jpg', '975158_614.jpg', '975163_614.jpg', '975164_614.jpg', '975168_614.jpg', '975173_614.jpg', '975181_614.jpg', '975188_614.jpg', '975189_614.jpg', '975193_614.jpg', '975194_614.jpg', '975196_614.jpg', '975198_614.jpg', '975202_614.jpg', '975204_614.jpg', '975207_614.jpg', '975208_614.jpg', '975211_614.jpg', '975212_614.jpg', '975214_614.jpg', '975452_614.jpg', '975461_614.jpg', '975463_614.jpg', '975472_614.jpg', '975485_614.jpg', '975497_614.jpg', '975519_614.jpg', '975529_614.jpg', '976111_614.jpg', '976116_614.jpg', '976117_614.jpg', '976123_614.jpg', '976682_614.jpg', '976691_614.jpg', '976699_614.jpg', '977029_614.jpg', '977090_614.jpg', '977091_614.jpg', '977282_614.jpg', '977561_614.jpg', '977762_614.jpg', '977767_614.jpg', '977772_614.jpg', '977777_614.jpg', '977780_614.jpg', '977782_614.jpg', '977788_614.jpg', '977801_614.jpg', '977893_614.jpg', '978119_614.jpg', '978120_614.jpg', '978123_614.jpg', '978710_614.jpg', '978716_614.jpg', '979530_614.jpg', '980067_614.jpg', '980172_614.jpg', '980523_614.jpg', '980533_614.jpg', '980546_614.jpg', '980651_614.jpg', '980706_614.jpg', '980750_614.jpg', '981005_614.jpg', '981049_614.jpg', '981084_614.jpg', '981500_614.jpg', '981504_614.jpg', '981789_614.jpg', '981829_614.jpg', '982145_614.jpg', '985416_614.jpg', '985423_614.jpg', '985437_614.jpg', '985891_614.jpg', '985898_614.jpg', '987256_614.jpg', '987304_614.jpg', '987313_614.jpg', '988010_614.jpg', '988024_614.jpg', '988077_614.jpg', '988116_614.jpg', '988672_614.jpg', '988688_614.jpg', '988696_614.jpg', '988704_614.jpg', '989655_614.jpg', '989827_614.jpg', '989840_614.jpg', '989862_614.jpg', '989871_614.jpg', '989879_614.jpg', '989901_614.jpg', '990024_614.jpg', '990031_614.jpg', '990049_614.jpg', '990067_614.jpg', '990072_614.jpg', '990078_614.jpg', '990187_614.jpg', '990538_614.jpg', '990552_614.jpg', '990554_614.jpg', '992456_614.jpg', '993570_614.jpg', '993964_614.jpg', '993999_614.jpg', '994000_614.jpg', '994145_614.jpg', '994147_614.jpg', '994149_614.jpg', '994247_614.jpg', '994255_614.jpg', '995360_614.jpg', '995371_614.jpg', '995564_614.jpg', '995766_614.jpg', '996060_614.jpg', '997570_614.jpg', '997673_614.jpg', '997678_614.jpg', '997747_614.jpg', '997880_614.jpg', '997944_614.jpg', '998611_614.jpg', '998643_614.jpg', '999137_614.jpg', '999182_614.jpg', '999836_614.jpg']



# JSON 데이터에서 원본 이미지 폴더에 없는 파일 정보 제거
for file in missing_in_images:
    if file in json_data:
        del json_data[file]

# 수정된 JSON 파일을 새 파일로 저장
new_json_file_path = r"C:\Users\LMK\Desktop\merge_10class\merged_class_10_updated.json"
try:
    with open(new_json_file_path, 'w') as new_json_file:
        json.dump(json_data, new_json_file, indent=4)
    print(f"수정된 JSON 파일이 '{new_json_file_path}'에 저장되었습니다.")
except Exception as e:
    print(f"파일을 저장하는 중에 오류가 발생했습니다: {e}")


원본 JSON 파일의 백업이 'C:\Users\LMK\Desktop\backup\merged_class_10_backup.json'에 저장되었습니다.
수정된 JSON 파일이 'C:\Users\LMK\Desktop\merge_10class\merged_class_10_updated.json'에 저장되었습니다.


In [1]:
### via에서 train, val 추출

In [ ]:
import os
import json
import random
from collections import defaultdict, Counter

# 설정 경로
via_file = r"C:\Users\LMK\Desktop\merge_10class\merged_class_10_updated.json" # VIA 파일 경로
output_folder = r"C:\Users\LMK\Desktop\merge_10class" # 데이터셋 분할 결과 저장 폴더

# 클래스별 목표 개수
target_train_size = 1200
target_val_size = 300
target_test_size = 20  # 테스트 세트 목표 개수

# 클래스별 최대 개수 설정 (실제 데이터 개수 고려)
class_target_sizes = {
    'Road_No_Stopping_or_Parking': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'Crosswalk': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'Road_Speed_Limit_in_School_Zone': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'Road_School_Zone': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'Road_No_Stopping_Zone': {'train': 455, 'val': 100, 'test': 10},
    'Road_No_Parking': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'no_parking': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'school_zone': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'stop': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'fire_hydrant': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size}
}

# 클래스별 이미지 목록
class_images = defaultdict(list)

# VIA 파일 읽기
try:
    with open(via_file, 'r', encoding='utf-8') as f:
        via_data = json.load(f)

    # 이미지별 클래스 수집
    for image_id, image_info in via_data.items():
        regions = image_info.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('label')
            if category:
                class_images[category].append(image_id)
except Exception as e:
    print(f"Error reading VIA file: {e}")

# train, val, test 세트
train_set = set()
val_set = set()
test_set = set()

# 클래스별로 train, val, test로 나누기
for category, images in class_images.items():
    random.shuffle(images)
    target_sizes = class_target_sizes.get(category, {'train': target_train_size, 'val': target_val_size, 'test': target_test_size})
    train_count = min(target_sizes['train'], len(images))  # 데이터 부족 시 최대한 할당
    val_count = min(target_sizes['val'], len(images) - train_count)
    test_count = min(target_sizes['test'], len(images) - train_count - val_count)

    # 현재 카테고리에서 사용될 이미지 필터링
    filtered_images = [img for img in images if img not in train_set and img not in val_set and img not in test_set]

    # train, val, test 세트로 나누기
    train_images = filtered_images[:train_count]
    val_images = filtered_images[train_count:train_count + val_count]
    test_images = filtered_images[train_count + val_count:train_count + val_count + test_count]

    train_set.update(train_images)
    val_set.update(val_images)
    test_set.update(test_images)

# 중복 제거
val_set.difference_update(train_set)
test_set.difference_update(train_set)
test_set.difference_update(val_set)

# 각 세트에 대한 통계 계산
def calculate_statistics(data_set, data_name):
    total_images = len(data_set)
    class_counts = Counter()

    for image_id in data_set:
        image_info = via_data[image_id]
        regions = image_info.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('label')
            if category:
                class_counts[category] += 1

    stats_output = {
        'total_images': total_images,
        'class_counts': dict(class_counts)
    }

    stats_file = os.path.join(output_folder, f'{data_name}_stats.json')
    with open(stats_file, 'w', encoding='utf-8') as f:
        json.dump(stats_output, f, indent=4, ensure_ascii=False)

    print(f"{data_name.capitalize()} set statistics saved to {stats_file}")
    return stats_output

# 통계 출력 및 저장
train_stats = calculate_statistics(train_set, 'train')
val_stats = calculate_statistics(val_set, 'val')
test_stats = calculate_statistics(test_set, 'test')

# 결과 출력
print(f"Train set size: {train_stats['total_images']}")
print(f"Validation set size: {val_stats['total_images']}")
print(f"Test set size: {test_stats['total_images']}")

# 분할된 데이터셋 저장
os.makedirs(output_folder, exist_ok=True)

train_file = os.path.join(output_folder, 'train.json')
val_file = os.path.join(output_folder, 'val.json')
test_file = os.path.join(output_folder, 'test.json')

# 저장할 데이터 생성
train_data = {image_id: via_data[image_id] for image_id in train_set}
val_data = {image_id: via_data[image_id] for image_id in val_set}
test_data = {image_id: via_data[image_id] for image_id in test_set}

try:
    with open(train_file, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, indent=4, ensure_ascii=False)
    with open(val_file, 'w', encoding='utf-8') as f:
        json.dump(val_data, f, indent=4, ensure_ascii=False)
    with open(test_file, 'w', encoding='utf-8') as f:
        json.dump(test_data, f, indent=4, ensure_ascii=False)

    print(f"Train set saved to {train_file}")
    print(f"Validation set saved to {val_file}")
    print(f"Test set saved to {test_file}")
except Exception as e:
    print(f"Error saving dataset: {e}")


Train set statistics saved to C:\Users\LMK\Desktop\merge_10class\train_stats.json
Val set statistics saved to C:\Users\LMK\Desktop\merge_10class\val_stats.json
Test set statistics saved to C:\Users\LMK\Desktop\merge_10class\test_stats.json
Train set size: 8814
Validation set size: 1637
Test set size: 97
Train set saved to C:\Users\LMK\Desktop\merge_10class\train.json
Validation set saved to C:\Users\LMK\Desktop\merge_10class\val.json
Test set saved to C:\Users\LMK\Desktop\merge_10class\test.json


In [ ]:
# 중복확인

import json

# 파일 경로 설정
train_via_file = r"C:\Users\LMK\Desktop\merge_10class\train.json"
val_via_file = r"C:\Users\LMK\Desktop\merge_10class\test.json"

# VIA 파일 읽기 함수
def load_via_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# 이미지 파일 이름 추출 함수
def extract_image_names(via_data):
    return set(via_data.keys())

# VIA 파일 읽기
train_via_data = load_via_file(train_via_file)
val_via_data = load_via_file(val_via_file)

# 이미지 파일 이름 목록 추출
train_image_names = extract_image_names(train_via_data)
val_image_names = extract_image_names(val_via_data)

# val 이미지가 train에 있는지 확인
common_images = val_image_names.intersection(train_image_names)
unique_val_images = val_image_names - train_image_names

print(f"Common images found in both 'val' and 'train': {len(common_images)}")
for image in common_images:
    print(f"Image {image} is present in both 'val' and 'train'.")

print(f"Unique images in 'val': {len(unique_val_images)}")
for image in unique_val_images:
    print(f"Image {image} is not present in 'train'.")


Common images found in both 'val' and 'train': 0
Unique images in 'val': 97
Image 0302_night_clear_smooth_08900847.png is not present in 'train'.
Image 0101_light_clear_traffic_06000147.png is not present in 'train'.
Image 1159573_614.jpg is not present in 'train'.
Image MP_KSC_020325.jpg is not present in 'train'.
Image 0302_night_clear_smooth_07022232.png is not present in 'train'.
Image 0302_night_clear_smooth_07031964.png is not present in 'train'.
Image 0302_light_rainy_traffic_03000137.png is not present in 'train'.
Image 1075107_614.jpg is not present in 'train'.
Image 0102_light_rainy_smooth_11000108.png is not present in 'train'.
Image MP_SEL_087277.jpg is not present in 'train'.
Image 0102_light_clear_smooth_06011039.png is not present in 'train'.
Image MP_SEL_082837.jpg is not present in 'train'.
Image 0102_light_clear_smooth_06010299.png is not present in 'train'.
Image 0302_night_clear_smooth_09006251.png is not present in 'train'.
Image MP_SEL_079831.jpg is not present in

In [ ]:
#통계



import os
import json
from collections import defaultdict

# 입력 및 출력 파일 경로 설정
input_via_file = r"C:\Users\LMK\Desktop\merge_10class\val.json" # 변환된 VIA 파일 경로
stats_output_file = r"C:\Users\LMK\Desktop\merge_10class\val_stats.json" # 통계 결과를 저장할 경로

# 카테고리별 통계 초기화
category_stats = defaultdict(int)
total_images = 0

# VIA 파일 읽기
try:
    with open(input_via_file, 'r', encoding='utf-8') as via_file:
        via_data = json.load(via_file)

    # 총 이미지 수 계산
    total_images = len(via_data)

    # 카테고리별 통계 계산
    for image_data in via_data.values():
        regions = image_data.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('label', 'Unknown')  ##label과 CATEGORY
            category_stats[category] += 1

    # 통계 결과 출력
    print(f"Total number of images: {total_images}")
    print("Category statistics:")
    for category, count in category_stats.items():
        print(f"{category}: {count}")

    # 통계 결과를 JSON 파일로 저장
    with open(stats_output_file, 'w', encoding='utf-8') as stats_file:
        json.dump({
            "total_images": total_images,
            "category_statistics": category_stats
        }, stats_file, indent=4, ensure_ascii=False)

    print(f"Statistics saved to {stats_output_file}")

except Exception as e:
    print(f"Error processing VIA file: {e}")


Total number of images: 1722
Category statistics:
Road_No_Parking: 450
fire_hydrant: 240
Road_No_Stopping_Zone: 97
Crosswalk: 568
no_parking: 324
stop: 309
Road_No_Stopping_or_Parking: 866
Road_School_Zone: 142
Road_Speed_Limit_in_School_Zone: 103
traffic_lane_yellow_solid: 250
Statistics saved to C:\Users\LMK\Desktop\merge_10class\val_stats.json


### json에 따라 이미지 가져오기

In [ ]:
import os
import json
import shutil

# 설정 경로
train_json_file = r"C:\Users\LMK\Desktop\merge_10class\train.json" # train.json 경로
val_json_file = r"C:\Users\LMK\Desktop\merge_10class\val.json" # val.json 경로
source_folder = r"C:\Users\LMK\Desktop\merge" # 원본 이미지가 있는 폴더
train_output_folder = r"C:\Users\LMK\Desktop\merge_10class\train" # train 이미지 저장할 폴더
val_output_folder = r"C:\Users\LMK\Desktop\merge_10class\val"  # val 이미지 저장할 폴더

# 폴더 생성
os.makedirs(train_output_folder, exist_ok=True)
os.makedirs(val_output_folder, exist_ok=True)

def copy_images(json_file, output_folder):
    try:
        # JSON 파일 읽기
        with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # 이미지 파일 복사
        for image_data in data.values():
            filename = image_data['filename']
            source_path = os.path.join(source_folder, filename)
            destination_path = os.path.join(output_folder, filename)

            if os.path.exists(source_path):
                shutil.copy(source_path, destination_path)
                print(f"Copied {filename} to {output_folder}")
            else:
                print(f"Image file {filename} not found in source folder.")

    except Exception as e:
        print(f"Error processing {json_file}: {e}")

# 이미지 파일 복사
copy_images(train_json_file, train_output_folder)
copy_images(val_json_file, val_output_folder)


Copied MP_KSC_009747.jpg to C:\Users\LMK\Desktop\merge_10class\train
Copied 0102_night_clear_smooth_10001559.png to C:\Users\LMK\Desktop\merge_10class\train
Copied 1812419_614.jpg to C:\Users\LMK\Desktop\merge_10class\train
Copied 0102_night_clear_smooth_01002210.png to C:\Users\LMK\Desktop\merge_10class\train
Copied MP_SEL_113874.jpg to C:\Users\LMK\Desktop\merge_10class\train
Copied 1538831_614.jpg to C:\Users\LMK\Desktop\merge_10class\train
Copied MP_SEL_076270.jpg to C:\Users\LMK\Desktop\merge_10class\train
Copied MP_SEL_074957.jpg to C:\Users\LMK\Desktop\merge_10class\train
Copied 1224649_614.jpg to C:\Users\LMK\Desktop\merge_10class\train
Copied 0102_light_clear_smooth_06011171.png to C:\Users\LMK\Desktop\merge_10class\train
Copied MP_KSC_009753.jpg to C:\Users\LMK\Desktop\merge_10class\train
Copied 0102_light_clear_smooth_11000480.png to C:\Users\LMK\Desktop\merge_10class\train
Copied 0402_light_clear_smooth_07000998.png to C:\Users\LMK\Desktop\merge_10class\train
Copied MP_SEL_

In [ ]:
### 이미지와 json확인

import os
import json

# 사용자 지정 경로
image_folder_path = r"C:\Users\LMK\Desktop\merge_10class\val"   # 이미지 폴더 경로
json_file_path = r"C:\Users\LMK\Desktop\merge_10class\val.json"   # JSON 파일 경로

# 이미지 파일 목록 가져오기
image_files = [f for f in os.listdir(image_folder_path) if os.path.isfile(os.path.join(image_folder_path, f))]

# JSON 파일 읽기
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# JSON 파일에서 이미지 이름 확인
json_image_names = set(item['filename'] for item in data.values())

# JSON에 없는 이미지 파일 목록 추출
missing_images = [img for img in image_files if img not in json_image_names]

# 결과 출력
if missing_images:
    print("JSON 파일에 없는 이미지 파일들:")
    for img in missing_images:
        print(img)
else:
    print("모든 이미지 파일이 JSON 파일에 존재합니다.")


모든 이미지 파일이 JSON 파일에 존재합니다.


In [ ]:
### via json에서 라벨 이미지정보 추출

import os
import json

# 설정 경로
via_file = r"C:\Users\LMK\Desktop\multiclass_last2\train\via.json"  # VIA 파일 경로
output_file = r"C:\Users\LMK\Desktop\merge_10class\train_filtered_traffic_lane_yellow_solid.json"  # 결과 저장 파일 경로

# 원하는 라벨
desired_label = "traffic_lane_yellow_solid"

# 필터링된 데이터 저장용 딕셔너리
filtered_data = {}

# VIA 파일 읽기
try:
    with open(via_file, 'r', encoding='utf-8') as f:
        via_data = json.load(f)

    # 이미지별로 데이터 필터링
    for image_id, image_info in via_data.items():
        regions = image_info.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('label')
            if category == desired_label:
                filtered_data[image_id] = image_info
                break  # 하나의 라벨만 필요하므로, 일치하는 라벨을 찾으면 중단
except Exception as e:
    print(f"Error reading VIA file: {e}")

# 결과 저장
try:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(filtered_data, f, indent=4, ensure_ascii=False)
    print(f"Filtered data saved to {output_file}")
except Exception as e:
    print(f"Error saving filtered data: {e}")


Filtered data saved to C:\Users\LMK\Desktop\merge_10class\train_filtered_traffic_lane_yellow_solid.json


### 재시각화

In [ ]:
import os
import json
import cv2
import numpy as np

# 입력 및 출력 경로 설정
via_file = r"C:\Users\LMK\Desktop\multi_V8\aug\via_merged.json"  # 변환된 VIA 파일 경로
image_folder = r"C:\Users\LMK\Desktop\multi_V8\aug\train\images"  # 원본 이미지 파일이 들어있는 폴더
output_folder = r"C:\Users\LMK\Desktop\multi_V8\aug\visualized"  # 시각화된 이미지를 저장할 폴더

# 색상 및 폰트 설정
color_mapping = {
    'fire_hydrant': (255, 0, 0),            # 빨간색
    'car': (0, 255, 0),                     # 녹색
    'truck': (0, 0, 255),                   # 파란색
    'stop': (255, 255, 0),                  # 노란색
    'motorcycle': (255, 165, 0),            # 주황색
    'traffic_lane_yellow_solid': (255, 255, 255),  # 흰색
    'school_zone': (255, 0, 255),           # 보라색
    'no_parking': (0, 255, 255),            # 청록색
    'Road_Speed_Limit_in_School_Zone': (128, 128, 0),  # 올리브색
    'Road_School_Zone': (75, 0, 130),       # 인디고색
    'Crosswalk': (0, 128, 128),             # 청록색
    'Road_No_Stopping_Zone': (128, 0, 128), # 자홍색
    'Road_No_Parking': (255, 69, 0),        # 오렌지 레드
    'Road_No_Stopping_or_Parking': (0, 128, 0) # 초록색
}

font_color = (255, 255, 255)  # 흰색
font_scale = 0.5
thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX

# VIA 파일 읽기
try:
    with open(via_file, 'r', encoding='utf-8') as via_json_file:
        via_data = json.load(via_json_file)

    # 폴더 생성
    os.makedirs(output_folder, exist_ok=True)

    # 이미지 파일 탐색 및 시각화
    for image_key, image_data in via_data.items():
        image_filename = image_data['filename']
        image_path = os.path.join(image_folder, image_filename)

        if not os.path.exists(image_path):
            print(f"Image file not found: {image_path}")
            continue

        # 이미지 읽기
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to read image: {image_path}")
            continue

        # 객체 시각화
        regions = image_data.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('label', 'Unknown')
            if category not in color_mapping:
                print(f"Unknown category: {category}")
                continue

            color = color_mapping[category]
            all_points_x = region['shape_attributes'].get('all_points_x', [])
            all_points_y = region['shape_attributes'].get('all_points_y', [])

            if all_points_x and all_points_y and len(all_points_x) == len(all_points_y):
                points = np.array([[x, y] for x, y in zip(all_points_x, all_points_y)], dtype=np.int32)
                points = points.reshape((-1, 1, 2))

                # 폴리곤 그리기
                image = cv2.polylines(image, [points], isClosed=True, color=color, thickness=thickness)

                # 텍스트 표시 (카테고리 이름)
                text_position = (int(all_points_x[0]), int(all_points_y[0]) - 10)
                cv2.putText(image, category, text_position, font, font_scale, font_color, thickness, cv2.LINE_AA)

        # 결과 이미지 저장
        output_image_path = os.path.join(output_folder, image_filename)
        cv2.imwrite(output_image_path, image)
        print(f"Saved visualized image to {output_image_path}")

except Exception as e:
    print(f"Error processing VIA file: {e}")


Saved visualized image to C:\Users\LMK\Desktop\multi_V8\aug\visualized\0102_light_clear_smooth_04006223.png
Saved visualized image to C:\Users\LMK\Desktop\multi_V8\aug\visualized\0102_light_clear_smooth_02004111.png
Saved visualized image to C:\Users\LMK\Desktop\multi_V8\aug\visualized\0302_light_clear_smooth_07001366.png
Saved visualized image to C:\Users\LMK\Desktop\multi_V8\aug\visualized\0302_night_clear_smooth_07022198.png
Saved visualized image to C:\Users\LMK\Desktop\multi_V8\aug\visualized\0302_night_clear_smooth_07018227.png
Saved visualized image to C:\Users\LMK\Desktop\multi_V8\aug\visualized\0102_light_clear_smooth_09006817.png
Saved visualized image to C:\Users\LMK\Desktop\multi_V8\aug\visualized\0102_light_foggy_smooth_07000184.png
Saved visualized image to C:\Users\LMK\Desktop\multi_V8\aug\visualized\0102_night_clear_smooth_11000826.png
Saved visualized image to C:\Users\LMK\Desktop\multi_V8\aug\visualized\0102_light_clear_smooth_06010689.png
Saved visualized image to C:

KeyboardInterrupt: 

In [ ]:
##카테고리 레이블


import os
import json

# 입력 및 출력 파일 경로 설정
input_via_file = r"C:\Users\LMK\Desktop\6class_via_polyline\via_format_polyline.json" # 변환할 VIA 파일 경로
output_via_file = r"C:\Users\LMK\Desktop\6class_via_polyline\via_format_polyline.json"  # 변환 결과를 저장할 경로

def convert_category_to_label(input_file, output_file):
    try:
        # VIA 파일 읽기
        with open(input_file, 'r', encoding='utf-8') as file:
            via_data = json.load(file)

        # 카테고리를 레이블로 변환
        for image_id, image_data in via_data.items():
            regions = image_data.get('regions', {})
            for region in regions.values():
                region_attributes = region.get('region_attributes', {})
                if 'category' in region_attributes:
                    region_attributes['label'] = region_attributes.pop('category')

        # 변환된 데이터 저장
        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(via_data, file, indent=4, ensure_ascii=False)

        print(f"Category to label conversion saved to {output_file}")

    except Exception as e:
        print(f"Error processing file {input_file}: {e}")

# 사용 예시
convert_category_to_label(input_via_file, output_via_file)


### 이미지 찾기

In [ ]:
import json
import re

# 사용자 지정 경로
json_file_path = r"C:\Users\LMK\Desktop\multiclass_last2\train\train.json"  # 원본 JSON 파일 경로
output_json_path = r"C:\Users\LMK\Desktop\multiclass_last2\train\filtered_train.json"  # 저장할 JSON 파일 경로

# JSON 파일 읽기
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 숫자 라벨 확인 함수
def has_numeric_label(region_attributes):
    # 모든 라벨이 숫자인지를 검사
    return any(re.match(r'^\d+$', str(label)) for label in region_attributes.values())

# 숫자 라벨을 가진 이미지를 제외한 데이터 필터링
filtered_data = {}
for key, value in data.items():
    if 'regions' in value:
        filtered_regions = {}
        for region_id, region in value['regions'].items():
            if 'region_attributes' in region and not has_numeric_label(region['region_attributes']):
                filtered_regions[region_id] = region
        if filtered_regions:
            filtered_data[key] = {
                'filename': value['filename'],
                'size': value['size'],
                'regions': filtered_regions
            }

# 수정된 JSON 파일 저장
with open(output_json_path, 'w', encoding='utf-8') as file:
    json.dump(filtered_data, file, indent=4, ensure_ascii=False)

print(f"Filtered data saved to '{output_json_path}'.")


In [ ]:
import os
import json
import shutil

# 설정 경로
via_file = r"C:\Users\LMK\Desktop\dataset_split2\val.json" # VIA 파일 경로
source_folder = r"C:\Users\LMK\Desktop\merge_val"  # 원본 이미지가 있는 폴더
output_folder = r"C:\Users\LMK\Desktop\multiclass\val" # 이미지 저장할 폴더

# 폴더 생성
os.makedirs(output_folder, exist_ok=True)

def copy_via_images(via_file, source_folder, output_folder):
    try:
        # VIA 파일 읽기
        with open(via_file, 'r', encoding='utf-8') as file:
            via_data = json.load(file)

        # 파일 목록 추출 및 복사
        for image_data in via_data.values():
            filename = image_data['filename']
            source_path = os.path.join(source_folder, filename)
            destination_path = os.path.join(output_folder, filename)

            if os.path.exists(source_path):
                shutil.copy(source_path, destination_path)
                print(f"Copied {filename} to {output_folder}")
            else:
                print(f"Image file {filename} not found in source folder.")

    except Exception as e:
        print(f"Error processing {via_file}: {e}")

# 사용 예시
copy_via_images(via_file, source_folder, output_folder)


In [ ]:
## 폴리곤 수정

import json
import numpy as np

# JSON 파일 경로
json_file_path = r"C:\Users\LMK\Desktop\multiclass_last2\val\val.json"
output_json_path = r"C:\Users\LMK\Desktop\multiclass_last2\modified_val.json"

# JSON 파일 읽기
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 좌표 정렬 함수
def sort_polygon(points):
    # 중심점을 기준으로 각도를 계산하여 정렬
    center = np.mean(points, axis=0)
    angles = np.arctan2(points[:, 1] - center[1], points[:, 0] - center[0])
    sorted_points = points[np.argsort(angles)]
    return sorted_points

# MP로 시작하는 이미지의 폴리곤 좌표 수정
for image_key, image_data in data.items():
    if image_key.startswith('MP'):
        regions = image_data.get('regions', {})
        for region_id, region in regions.items():
            shape_attributes = region['shape_attributes']
            if 'all_points_x' in shape_attributes and 'all_points_y' in shape_attributes:
                all_points_x = shape_attributes['all_points_x']
                all_points_y = shape_attributes['all_points_y']

                if len(all_points_x) == len(all_points_y) and len(all_points_x) == 4:
                    points = np.array(list(zip(all_points_x, all_points_y)))

                    # 폴리곤 좌표가 교차하는 경우 수정
                    sorted_points = sort_polygon(points)
                    shape_attributes['all_points_x'] = sorted_points[:, 0].tolist()
                    shape_attributes['all_points_y'] = sorted_points[:, 1].tolist()

# 수정된 JSON 파일 저장
with open(output_json_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

print(f"Modified JSON saved to '{output_json_path}'")


In [ ]:
### json 빈 좌표 검증
import json

# JSON 파일 경로
json_file_path = r"C:\Users\LMK\Desktop\6class_via_polyline\via_format_polyline.json"

# JSON 파일 읽기
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 좌표가 없는 데이터를 찾기 위한 함수
def find_empty_coordinates(data):
    images_with_empty_coords = []

    for image_key, image_info in data.items():
        for region_key, region_info in image_info.get('regions', {}).items():
            shape_attributes = region_info.get('shape_attributes', {})
            all_points_x = shape_attributes.get('all_points_x', [])
            all_points_y = shape_attributes.get('all_points_y', [])

            if not all_points_x or not all_points_y:
                images_with_empty_coords.append({
                    'image': image_key,
                    'region': region_key,
                    'filename': image_info.get('filename', 'unknown'),
                    'all_points_x': all_points_x,
                    'all_points_y': all_points_y
                })

    return images_with_empty_coords

# 좌표가 없는 데이터 찾기
empty_coords = find_empty_coordinates(data)

# 결과 출력
if empty_coords:
    print("좌표가 없는 항목을 찾았습니다:")
    for item in empty_coords:
        print(f"이미지: {item['image']}, 파일명: {item['filename']}, 영역: {item['region']}, "
              f"all_points_x: {item['all_points_x']}, all_points_y: {item['all_points_y']}")
else:
    print("모든 항목에 좌표가 있습니다.")


In [ ]:
#빈 좌표 제거

import json

# JSON 파일 경로
json_file_path = r"C:\Users\LMK\Desktop\6class_via_polyline\via_format_polyline.json"

# JSON 파일 읽기
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 좌표가 없는 객체를 삭제하는 함수
def remove_empty_coordinates(data):
    for image_key, image_info in data.items():
        if 'regions' in image_info:
            regions = image_info['regions']
            for region_key in list(regions.keys()):  # keys() 리스트를 만들어야 삭제 가능
                shape_attributes = regions[region_key].get('shape_attributes', {})
                all_points_x = shape_attributes.get('all_points_x', [])
                all_points_y = shape_attributes.get('all_points_y', [])

                if not all_points_x or not all_points_y:
                    # 좌표가 없는 객체 삭제
                    del regions[region_key]

            # 만약 모든 regions가 삭제되었다면, 'regions' 키를 제거
            if not regions:
                del image_info['regions']

    return data

# 좌표가 없는 객체 삭제
cleaned_data = remove_empty_coordinates(data)

# 수정된 JSON 파일 저장
cleaned_json_file_path = r"C:\Users\LMK\Desktop\6class_via_polyline\via_format_polyline_cleaned.json"
with open(cleaned_json_file_path, 'w', encoding='utf-8') as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

print(f"좌표가 없는 객체가 제거된 JSON 파일이 '{cleaned_json_file_path}'에 저장되었습니다.")


In [ ]:
### 좌표 다각형 검증
import json

# JSON 파일 경로
json_file_path = r'C:/Users/LMK/Desktop/multiclass_last2/val/preprocessed_via.json'

# JSON 파일 읽기
with open(json_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 좌표가 유효한지 확인하는 함수
def is_valid_polygon(points_x, points_y):
    if len(points_x) < 3 or len(points_y) < 3:
        return False
    if len(set(points_x)) < 3 or len(set(points_y)) < 3:
        return False
    return True

# 유효하지 않은 좌표 데이터를 찾기 위한 함수
def find_invalid_coordinates(data):
    invalid_coords = []

    for image_key, image_info in data.items():
        for region_key, region in image_info.get('regions', {}).items():
            shape_attributes = region.get('shape_attributes', {})
            points_x = shape_attributes.get('all_points_x', [])
            points_y = shape_attributes.get('all_points_y', [])

            if not is_valid_polygon(points_x, points_y):
                invalid_coords.append({
                    'image': image_key,
                    'region': region_key,
                    'filename': image_info.get('filename', 'unknown'),
                    'all_points_x': points_x,
                    'all_points_y': points_y
                })

    return invalid_coords

# 유효하지 않은 좌표 데이터 찾기
invalid_coords = find_invalid_coordinates(data)

# 결과 출력
if invalid_coords:
    print("유효하지 않은 좌표 항목을 찾았습니다:")
    for item in invalid_coords:
        print(f"이미지: {item['image']}, 파일명: {item['filename']}, 영역: {item['region']}, "
              f"all_points_x: {item['all_points_x']}, all_points_y: {item['all_points_y']}")
else:
    print("모든 항목이 유효한 좌표를 가지고 있습니다.")


In [ ]:
## 폴리라인 폴리곤 변화 밑 폐곡선

import json
import os

data_path = r"C:/Users/LMK/Desktop/multiclass_last2/val/"
input_json_file = os.path.join(data_path, 'val.json')
output_json_file = os.path.join(data_path, 'preprocessed_via.json')

# 폴리라인을 바운딩 박스로 변환
def polyline_to_bbox(points_x, points_y):
    x_min = min(points_x)
    x_max = max(points_x)
    y_min = min(points_y)
    y_max = max(points_y)
    return x_min, y_min, x_max, y_max

# 데이터 전처리 함수
def preprocess_via_json(via_json):
    for image_id, image_data in via_json.items():
        for region_key, region_data in image_data.get('regions', {}).items():
            shape_attributes = region_data.get('shape_attributes', {})
            if shape_attributes.get('name') == 'polyline':
                all_points_x = shape_attributes.get('all_points_x', [])
                all_points_y = shape_attributes.get('all_points_y', [])

                if len(all_points_x) < 2 or len(all_points_y) < 2:
                    continue  # 유효하지 않은 폴리라인 무시

                # 폴리라인을 바운딩 박스로 변환
                x_min, y_min, x_max, y_max = polyline_to_bbox(all_points_x, all_points_y)

                shape_attributes['name'] = 'polygon'
                shape_attributes['all_points_x'] = [x_min, x_max, x_max, x_min, x_min]
                shape_attributes['all_points_y'] = [y_min, y_min, y_max, y_max, y_min]

    return via_json

# 파일 읽기 및 전처리
with open(input_json_file, 'r') as file:
    via_data = json.load(file)

preprocessed_via_data = preprocess_via_json(via_data)

# 결과 저장
with open(output_json_file, 'w') as file:
    json.dump(preprocessed_via_data, file, indent=2)

print(f"Preprocessed JSON saved to {output_json_file}")


### augmentation

In [ ]:
!pip install albumentations


In [ ]:
import os
import json
import random
from collections import defaultdict

# 설정 경로
via_file = r"E:\desktop\6class_merge\aug_schol_zone_crosswalk.json" # VIA 파일 경로
output_folder = r"C:\Users\LMK\Desktop\multi_V8\aug"  # 데이터셋 분할 결과 저장 폴더

# 클래스별 목표 개수
target_train_size = 100
target_val_size = 20

# 클래스별 최대 개수 설정 (실제 데이터 개수 고려)
class_target_sizes = {
    #'Road_No_Stopping_or_Parking': {'train': target_train_size, 'val': target_val_size},
    'Crosswalk': {'train': target_train_size, 'val': target_val_size},
    'Road_Speed_Limit_in_School_Zone': {'train': target_train_size, 'val': target_val_size},
    'Road_School_Zone': {'train': target_train_size, 'val': target_val_size},
    #'Road_No_Stopping_Zone': {'train': 450, 'val': 100},
    #'Road_No_Parking': {'train': target_train_size, 'val': target_val_size}
}

# 클래스별 이미지 목록
class_images = defaultdict(list)

# VIA 파일 읽기
try:
    with open(via_file, 'r', encoding='utf-8') as f:
        via_data = json.load(f)

    # 이미지별 클래스 수집
    for image_id, image_info in via_data.items():
        regions = image_info.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('category')
            if category:
                class_images[category].append(image_id)
except Exception as e:
    print(f"Error reading VIA file: {e}")

# train과 val 세트
train_set = set()
val_set = set()

# 클래스별로 train과 val로 나누기
for category, images in class_images.items():
    random.shuffle(images)
    target_sizes = class_target_sizes.get(category, {'train': target_train_size, 'val': target_val_size})
    train_count = target_sizes['train']
    val_count = target_sizes['val']

    # 현재 카테고리에서 사용될 이미지 필터링
    filtered_images = [img for img in images if img not in train_set and img not in val_set]

    # train과 val 세트로 나누기
    train_images = filtered_images[:train_count]
    val_images = filtered_images[train_count:train_count + val_count]

    train_set.update(train_images)
    val_set.update(val_images)

# 중복 제거
val_set.difference_update(train_set)  # train_set과 중복된 항목 제거

# 결과 출력
print(f"Train set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")

# 분할된 데이터셋 저장
os.makedirs(output_folder, exist_ok=True)

train_file = os.path.join(output_folder, 'train.json')
val_file = os.path.join(output_folder, 'val.json')

# 저장할 데이터 생성
train_data = {image_id: via_data[image_id] for image_id in train_set}
val_data = {image_id: via_data[image_id] for image_id in val_set}

try:
    with open(train_file, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, indent=4, ensure_ascii=False)

    with open(val_file, 'w', encoding='utf-8') as f:
        json.dump(val_data, f, indent=4, ensure_ascii=False)

    print(f"Train set saved to {train_file}")
    print(f"Validation set saved to {val_file}")
except Exception as e:
    print(f"Error saving dataset: {e}")


Train set size: 0
Validation set size: 0
Train set saved to C:\Users\LMK\Desktop\multi_V8\aug\train.json
Validation set saved to C:\Users\LMK\Desktop\multi_V8\aug\val.json


In [ ]:
import os
import json
import random
from collections import defaultdict, Counter

# 설정 경로
via_file = r"E:\desktop\6class_merge\aug_schol_zone_crosswalk.json" # VIA 파일 경로
output_folder = r"C:\Users\LMK\Desktop\multi_V8\aug"  # 데이터셋 분할 결과 저장 폴더

# 클래스별 목표 개수
target_train_size = 500
target_val_size = 300
target_test_size = 20  # 테스트 세트 목표 개수

# 클래스별 최대 개수 설정 (실제 데이터 개수 고려)
class_target_sizes = {
    #'Road_No_Stopping_or_Parking': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'Crosswalk': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'Road_Speed_Limit_in_School_Zone': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    'Road_School_Zone': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    #'Road_No_Stopping_Zone': {'train': 455, 'val': 100, 'test': 10},
    #'Road_No_Parking': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    #'no_parking': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    #'school_zone': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    #'stop': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size},
    #'fire_hydrant': {'train': target_train_size, 'val': target_val_size, 'test': target_test_size}
}

# 클래스별 이미지 목록
class_images = defaultdict(list)

# VIA 파일 읽기
try:
    with open(via_file, 'r', encoding='utf-8') as f:
        via_data = json.load(f)

    # 이미지별 클래스 수집
    for image_id, image_info in via_data.items():
        regions = image_info.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('label')
            if category:
                class_images[category].append(image_id)
except Exception as e:
    print(f"Error reading VIA file: {e}")

# train, val, test 세트
train_set = set()
val_set = set()
test_set = set()

# 클래스별로 train, val, test로 나누기
for category, images in class_images.items():
    random.shuffle(images)
    target_sizes = class_target_sizes.get(category, {'train': target_train_size, 'val': target_val_size, 'test': target_test_size})
    train_count = min(target_sizes['train'], len(images))  # 데이터 부족 시 최대한 할당
    val_count = min(target_sizes['val'], len(images) - train_count)
    test_count = min(target_sizes['test'], len(images) - train_count - val_count)

    # 현재 카테고리에서 사용될 이미지 필터링
    filtered_images = [img for img in images if img not in train_set and img not in val_set and img not in test_set]

    # train, val, test 세트로 나누기
    train_images = filtered_images[:train_count]
    val_images = filtered_images[train_count:train_count + val_count]
    test_images = filtered_images[train_count + val_count:train_count + val_count + test_count]

    train_set.update(train_images)
    val_set.update(val_images)
    test_set.update(test_images)

# 중복 제거
val_set.difference_update(train_set)
test_set.difference_update(train_set)
test_set.difference_update(val_set)

# 각 세트에 대한 통계 계산
def calculate_statistics(data_set, data_name):
    total_images = len(data_set)
    class_counts = Counter()

    for image_id in data_set:
        image_info = via_data[image_id]
        regions = image_info.get('regions', {})
        for region in regions.values():
            category = region['region_attributes'].get('label')
            if category:
                class_counts[category] += 1

    stats_output = {
        'total_images': total_images,
        'class_counts': dict(class_counts)
    }

    stats_file = os.path.join(output_folder, f'{data_name}_stats.json')
    with open(stats_file, 'w', encoding='utf-8') as f:
        json.dump(stats_output, f, indent=4, ensure_ascii=False)

    print(f"{data_name.capitalize()} set statistics saved to {stats_file}")
    return stats_output

# 통계 출력 및 저장
train_stats = calculate_statistics(train_set, 'train')
val_stats = calculate_statistics(val_set, 'val')
test_stats = calculate_statistics(test_set, 'test')

# 결과 출력
print(f"Train set size: {train_stats['total_images']}")
print(f"Validation set size: {val_stats['total_images']}")
print(f"Test set size: {test_stats['total_images']}")

# 분할된 데이터셋 저장
os.makedirs(output_folder, exist_ok=True)

train_file = os.path.join(output_folder, 'train.json')
val_file = os.path.join(output_folder, 'val.json')
test_file = os.path.join(output_folder, 'test.json')

# 저장할 데이터 생성
train_data = {image_id: via_data[image_id] for image_id in train_set}
val_data = {image_id: via_data[image_id] for image_id in val_set}
test_data = {image_id: via_data[image_id] for image_id in test_set}

try:
    with open(train_file, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, indent=4, ensure_ascii=False)
    with open(val_file, 'w', encoding='utf-8') as f:
        json.dump(val_data, f, indent=4, ensure_ascii=False)
    with open(test_file, 'w', encoding='utf-8') as f:
        json.dump(test_data, f, indent=4, ensure_ascii=False)

    print(f"Train set saved to {train_file}")
    print(f"Validation set saved to {val_file}")
    print(f"Test set saved to {test_file}")
except Exception as e:
    print(f"Error saving dataset: {e}")


Train set statistics saved to C:\Users\LMK\Desktop\multi_V8\aug\train_stats.json
Val set statistics saved to C:\Users\LMK\Desktop\multi_V8\aug\val_stats.json
Test set statistics saved to C:\Users\LMK\Desktop\multi_V8\aug\test_stats.json
Train set size: 975
Validation set size: 309
Test set size: 14
Train set saved to C:\Users\LMK\Desktop\multi_V8\aug\train.json
Validation set saved to C:\Users\LMK\Desktop\multi_V8\aug\val.json
Test set saved to C:\Users\LMK\Desktop\multi_V8\aug\test.json


In [ ]:
import json
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

def load_via_data(path):
    if os.path.exists(path):
        with open(path, 'r') as f:
            return json.load(f)
    else:
        return {}

def save_transformed_data(data, output_path):
    with open(output_path, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def clamp_keypoints(keypoints, width, height):
    clamped_keypoints = []
    for x, y in keypoints:
        clamped_x = max(0, min(x, width - 1))
        clamped_y = max(0, min(y, height - 1))
        clamped_keypoints.append((clamped_x, clamped_y))
    return clamped_keypoints

# 설정 경로
via_json_path = r"C:\Users\LMK\Desktop\multi_V8\aug\train.json"
output_dir = r"C:\Users\LMK\Desktop\multi_V8\aug\train\trans"
image_dir = r"C:\Users\LMK\Desktop\multi_V8\aug\train"  # 실제 이미지가 저장된 폴더 경로
os.makedirs(output_dir, exist_ok=True)

# JSON 파일 읽기
via_data = load_via_data(via_json_path)

# 데이터 파싱
annotations = []
for key, value in via_data.items():
    filename = value['filename']
    regions = value['regions']
    polygons = []
    labels = []
    for region_key, region_value in regions.items():
        shape_attributes = region_value['shape_attributes']
        all_points_x = shape_attributes['all_points_x']
        all_points_y = shape_attributes['all_points_y']
        polygon = [(x, y) for x, y in zip(all_points_x, all_points_y)]
        polygons.append(polygon)
        labels.append(region_value['region_attributes']['label'])
    annotations.append((filename, polygons, labels))

# 어그멘테이션 정의
transform = A.Compose([
    #A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),
    A.Resize(height=640, width=640, p=1.0),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# 변형된 이미지 저장 및 JSON 병합
merged_via_data = load_via_data(via_json_path)
all_transformed_data = {}

for iteration in range(3):
    print(f"Iteration {iteration + 1}")
    via_transformed_data = {}

    for filename, polygons, labels in annotations:
        image_path = os.path.join(image_dir, filename)

        if not os.path.exists(image_path):
            print(f"Error: File '{image_path}' does not exist.")
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Failed to read the image '{image_path}'.")
            continue

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, _ = image.shape

        keypoints = [point for polygon in polygons for point in polygon]
        clamped_keypoints = clamp_keypoints(keypoints, width, height)

        transformed = transform(image=image, keypoints=clamped_keypoints)
        transformed_image = transformed['image']
        transformed_keypoints = transformed['keypoints']

        transformed_polygons = []
        index = 0
        for polygon in polygons:
            num_points = len(polygon)
            transformed_polygon = transformed_keypoints[index:index + num_points]
            transformed_polygons.append(transformed_polygon)
            index += num_points

        transformed_filename = f"transformed_{iteration + 1}_{filename}"
        transformed_image_path = os.path.join(output_dir, transformed_filename)

        transformed_image_np = transformed_image.permute(1, 2, 0).cpu().numpy()
        transformed_image_np = np.clip(transformed_image_np * 255, 0, 255).astype(np.uint8)
        transformed_image_np = cv2.cvtColor(transformed_image_np, cv2.COLOR_RGB2BGR)
        cv2.imwrite(transformed_image_path, transformed_image_np)

        new_polygons = [{'all_points_x': [int(x) for x, y in polygon], 'all_points_y': [int(y) for x, y in polygon]} for polygon in transformed_polygons]

        new_key = f"{filename}_transformed_{iteration + 1}"
        via_transformed_data[new_key] = {
            'filename': transformed_filename,
            'regions': {str(i): {'shape_attributes': new_polygons[i], 'region_attributes': {'label': labels[i]}} for i in range(len(new_polygons))}
        }

    # 각 반복마다 변환된 데이터를 병합 데이터에 추가
    all_transformed_data.update(via_transformed_data)

    # 각 반복마다 변환된 데이터 저장
    transformed_output_path = os.path.join(output_dir, f'via_transformed_{iteration + 1}.json')
    save_transformed_data(via_transformed_data, transformed_output_path)

    print(f"Total processed images in iteration {iteration + 1}: {len(via_transformed_data)}")

# 모든 변환된 데이터를 병합
merged_via_data.update(all_transformed_data)

merged_output_path = os.path.join(output_dir, 'via_merged.json')
save_transformed_data(merged_via_data, merged_output_path)

print(f"Merged transformed data saved to {merged_output_path}")


Iteration 1
Total processed images in iteration 1: 975
Iteration 2
Total processed images in iteration 2: 975
Iteration 3
Total processed images in iteration 3: 975
Merged transformed data saved to C:\Users\LMK\Desktop\multi_V8\aug\train\trans\via_merged.json


In [ ]:
import os
import json
import shutil

# 설정 경로
train_json_file = r"C:\Users\LMK\Desktop\multi_V8\aug\train.json" # train.json 경로
val_json_file = r"C:\Users\LMK\Desktop\multi_V8\aug\val.json" # val.json 경로
source_folder = r"E:\desktop\merge" # 원본 이미지가 있는 폴더
train_output_folder = r"C:\Users\LMK\Desktop\multi_V8\aug\train" # train 이미지 저장할 폴더
val_output_folder = r"C:\Users\LMK\Desktop\multi_V8\aug\val"  # val 이미지 저장할 폴더

# 폴더 생성
os.makedirs(train_output_folder, exist_ok=True)
os.makedirs(val_output_folder, exist_ok=True)

def copy_images(json_file, output_folder):
    try:
        # JSON 파일 읽기
        with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # 이미지 파일 복사
        for image_data in data.values():
            filename = image_data['filename']
            source_path = os.path.join(source_folder, filename)
            destination_path = os.path.join(output_folder, filename)

            if os.path.exists(source_path):
                shutil.copy(source_path, destination_path)
                print(f"Copied {filename} to {output_folder}")
            else:
                print(f"Image file {filename} not found in source folder.")

    except Exception as e:
        print(f"Error processing {json_file}: {e}")

# 이미지 파일 복사
copy_images(train_json_file, train_output_folder)
copy_images(val_json_file, val_output_folder)


Copied 0102_light_clear_smooth_04006223.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0102_light_clear_smooth_02004111.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0302_light_clear_smooth_07001366.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0302_night_clear_smooth_07022198.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0302_night_clear_smooth_07018227.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0102_light_clear_smooth_09006817.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0102_light_foggy_smooth_07000184.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0102_night_clear_smooth_11000826.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0102_light_clear_smooth_06010689.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0402_night_clear_smooth_11000435.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0102_light_clear_smooth_09008067.png to C:\Users\LMK\Desktop\multi_V8\aug\train
Copied 0102_light_clear_smooth_02003823.png